# Brightway2 seminar
Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

## Day 1, AM:

#### Getting started:  
  - Setting up  
    * Downloading and installing Brightway  
    * Accessing Brightway
  - Projects  
    * Concept  
    * Contents  
    * Creation  
  - bw2_setup()  
    * biosphere3 database  
    * Methods  
    * Looking up elementary flows (list comprehensions, search)  
    * Searching for methods  
    * Nice display of data in methods 
  - LCI databases  
    - Importing (succinct)  
    - Activities, exchanges

#### Our first LCA - simplest case:  
  - The `bw.LCA` object  
  - Defining the functional unit  
  - `.lci()` method  
    * Demand array  
    * Indices and activity_dict, biosphere_dict and product_dict  
    * $A$ and $B$ matrices, (CSR, COO)  
    * Reverse dictionaries  
    * Nice layout of information in Pandas series  
  - `.lcia()` method  
  - Simple contribution analysis  
  
#### Our second LCA - comparative LCA:
  - Different approaches:  
    * Comparing two LCA objects  
    * Creating a "comparison" activity
    * MultiLCA 
  - Presenting the results
    
#### Our third LCA - Multiple LCIA impact categories
  - Different approaches:  
    * `for` loop  
    * MultiLCA   
    * Working directly with the characterization factors  
  - Presenting the results

## Getting started

Status (2017-02-22): 
  - As used in intro session at CIRAIG/CRVMR
  - Needs clean-up (e.g. header levels)
  - Need to remove ecoinvent v3.3 examples.
  - Need a rearead with rested head.

### Setting up

#### Installation

*Windows*  
The easiest way to install Brightway2 is to download the [Brightway2 Windows installation package](https://brightwaylca.org/data/bw2-python-windows.7z), and extract to the C:\ drive. This will create the directory C:\bw2-python\.  

*Mac, Linux*  
Please follow instructions [here](https://brightwaylca.org/dev-docs/installation.html). 

#### Accessing Brightway2 libraries

The different modules in Brightway2 are Python libraries. This means that, to use them, you can use any environment from which you normally use Python (Idle, command prompt, Spyder or, as is the case today, Jupyter Notebooks).  

We will favour Jupyter Notebooks in this seminar because it allows us to integrate code and text. It will also allow us to provide code snippets for you to complete.  

Note that the [Brightway2 installation package](https://docs.brightwaylca.org/installation.html) installs Brightway2 in a separate [Conda environment](https://conda.io/docs/using/using.html). This isolates Brightway2 from your other Python installations. It however requires you to activate the bw2 environment. You can do this the same way you [normally activate Conda environments](https://conda.io/docs/using/envs.html#change-environments-activate-deactivate), or by executing the bw2-env.bat batch file installed in your `bw2-python` directory (located at `C:\bw2-python`in Windows).  

The `bw2-python` directory also offers two other ways to run Brightway2: via IPython (run the `bw2-ipython.bat` file) or via Jupyter Notebooks (`bw2-notebook.bat`).  

For this course, you should run `bw2-notebook.bat` and open the Notebooks (such as this one), allowing you to directly run the code and get some hands-on experience. 

Like all other Python packages, you need to `import` Brightway2 modules. We will here import everything from Brightway2:

In [1]:
import brightway2 as bw

### Projects, set-up

The top-level containent in Brightway2 is the project (see [here](https://docs.brightwaylca.org/intro.html#projects) for a description and [here](https://docs.brightwaylca.org/technical/bw2data.html#projects) for the docs). A project contains LCI databases, LCIA methods and other less often used objects. Objects from one project do not interract with objects within other projects. By analogy, projects are like databases in openLCA and SimePro.  
When you first launch Brightway2, you will be in the `default` project. You can check this using the `current` property of the `projects` object: 

In [2]:
bw.projects.current

'default'

Let's create a new project for this seminar, unsurprisingly called "bw2_seminar_2017". There are two ways of doing this:  
* `projects.create_project('bw2_seminar_2017')` will create the project, but you will remain in your current project.
* `projects.set_current('bw2_seminar_2017')` will switch you to the project passed as argument, and create it first if it doesn't exist.  Let's do the latter:

In [3]:
# The name of the project is entered as string; 
# it doesn't really have any restrictions, so can include spaces, 
# special characters, other languages, or even emoji.

bw.projects.set_current('bw2_seminar_2017') 

You can always see what projects you have on your computer by running `list(bw.projects)`. Unless you have worked with Brightway2 before on your computer, your list should contain two projects: 'default' and 'bw2_seminar_2017'.

In [ ]:
# Exercise: list the projects on your computer.


Like in all Python modules, you can get additionnal information on the `projects` object and associated properties and methods by typing `help(projects)`. The [docs](https://docs.brightwaylca.org/technical/bw2data.html#projects) are of course more verbose.

One property of `projects` is its location, given by `projects.dir`:

In [4]:
bw.projects.dir

'C:\\Users\\pasca\\AppData\\Local\\pylca\\Brightway3\\bw2_seminar_2017.058cb6360ae51ecc25fe811e209c8b06'

Looking at what is inside:  
<img src="project_folder_before_setup.JPG">

All the directories are empty except the `lci` directory, which contains an empty database:

All in all, the project takes up 4KB.  
Its now time to start populating the project.

The first thing you should do is add flow and LCIA methods. This is done by running the `bw2setup` function:

In [6]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


The output tells us that bw2_setup created some very useful things:  
  - Created a database called "biosphere3": this database contains elementary flows (called biosphere exchanges in Brightway2)  
  - 718 impact assessment methods  
  
It also created some a `mapping` between the imported exchanges and some integer: more on this later.  
The whole directory now takes up 125MB.

Looking at the contents of the `databases.db` database, we see we have imported 4029 exchanges:  
<img src="database_after_setup_data.JPG">

While not impossible to interact with the data at this level, you probably never will unless you are developping some funky program. Instead, it is strongly recommended to learn to work with `abstractions`. Let's explore some now:

#### Biosphere database  
The data in Brightway is stored in databases. When you run `bw2_setup()`, the first database that is created is the 'biosphere' database, as mentionned above.  
You can always list the databases inside a project by simply typing 'databases'. This accesses the 'database.json' file in your 'project.dir' (I learned the latter by typing `databases?`, you should try it too.)

In [8]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 2.2
	ecoinvent 3.3 cutoff

To actually access a database, you need to use the `Database' method` (again, you can type `Database?` for more information - this is the last I'll mention this.

In [10]:
bw.Database('biosphere3')

Brightway2 SQLiteBackend: biosphere3

It doesn't actually return anything other than information about the Backend.  
However, there are many properties and functions associated with this database object.  These are found [here](https://docs.brightwaylca.org/technical/bw2data.html). We can also have a look through the autocomplete. Let's assign the database to a variable:

In [11]:
my_bio = bw.Database('biosphere3')

Let's check the my_bio `type`:

In [12]:
type(my_bio)

bw2data.backends.peewee.database.SQLiteBackend

Let's check its length:

In [13]:
len(my_bio)

4029

This is exactly the number of items we saw had been added to databases.db

If you type `my_bio.` and click on tab, you should get a list of properties and methods associated with database objects. Try this now:

In [ ]:
my_bio.        #Type my_bio. and click tab. Have a look at the different properties and objects

Some of the more basic ones we will be using now are :  
  - `random()` - returns a random activity in the database
  - `get(*valid_exchange_tuple*)` - returns an activity, but you must know the activity key
  - `load()` - loads the whole database as a dictionary.
  - `make_searchable` - allows searching of the database (by default, it is already searchable)
  - `search` - search the database  
  
Lets start with random:

In [14]:
my_bio.random()

'Imazamox' (kilogram, None, ('air', 'non-urban air or from high stacks'))

This returns a biosphere activity, but without assigning it to a variable, there is not much we can do with it directly.  
Let's assign it to a variable:

In [15]:
random_biosphere = my_bio.random()
random_biosphere

'Methane, dichloro-, HCC-30' (kilogram, None, ('water', 'surface water'))

In [16]:
type(random_biosphere)

bw2data.backends.peewee.proxies.Activity

To see what it contains, we can load convert it to a dictionary:

In [17]:
random_biosphere.as_dict()

{'categories': ('water', 'surface water'),
 'code': '1b6a35b8-f6c8-404d-9c0e-e09b8fdb2d9e',
 'database': 'biosphere3',
 'name': 'Methane, dichloro-, HCC-30',
 'type': 'emission',
 'unit': 'kilogram'}

We can see that the activities in the biosphere3 database have unique codes, which we can use with the `get` function:

In [18]:
my_bio.get(random_biosphere['code'])

'Methane, dichloro-, HCC-30' (kilogram, None, ('water', 'surface water'))

Activities can also be "gotten" via `get_activity`, but the argument is the activity **key**, consisting of a tuple with two elements: the database name, and the activity code.

In [19]:
bw.get_activity((                         #Enter a tuple consisting of the database name (as string) and activity code

'Aniline' (kilogram, None, ('air',))

You can always find (or return) the key to an activity using the `.key` property.

In [20]:
random_biosphere.key

('biosphere3', '1b6a35b8-f6c8-404d-9c0e-e09b8fdb2d9e')

Let's say we are looking for a specific elementary flow, we can use search:

In [21]:
bw.Database('biosphere3').search('carbon dioxide')

['Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil',)),
 'Carbon 

The database object is also iterable, allowing "home-made" searches through list comprehensions.

In [22]:
[act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'urban air' in str(act['categories'])
         ]

['Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))]

In [23]:
act_I_want = [act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'urban air' in str(act['categories'])
         ][0]

In [24]:
act_I_want.as_dict()['code']

'd6235194-e4e6-4548-bfa3-ac095131aef4'

In [ ]:
# Exercise: look for and assign to a variable an emission of nitrous oxide emitted to air in the "urban air" subcompartment.

Let's leave the biosphere database here for now.

#### Methods

bw2_setup() also installed LCIA methods.

In [25]:
bw.methods

Methods dictionary with 718 objects, including:
	('CML 2001', 'acidification potential', 'average European')
	('CML 2001', 'acidification potential', 'generic')
	('CML 2001', 'climate change', 'GWP 100a')
	('CML 2001', 'climate change', 'GWP 20a')
	('CML 2001', 'climate change', 'GWP 500a')
	('CML 2001', 'climate change', 'lower limit of net GWP')
	('CML 2001', 'climate change', 'upper limit of net GWP')
	('CML 2001', 'eutrophication potential', 'average European')
	('CML 2001', 'eutrophication potential', 'generic')
	('CML 2001', 'freshwater aquatic ecotoxicity', 'FAETP 100a')
Use `list(this object)` to get the complete list.

One can load a random method:

In [26]:
bw.methods.random()

('CML 2001 w/o LT', 'photochemical oxidation w/o LT', 'high NOx POCP w/o LT')

In [28]:
type(bw.methods.random())

tuple

Here, the random method returns the tuple by which the method is identified. To get to an actual method, the following syntax is used:

In [30]:
bw.Method(bw.methods.random())

Brightway2 Method: CML 2001: freshwater aquatic ecotoxicity: FAETP 100a

Of course, a random method is probably not useful except to play around. To find an actual method, one can again use list comprehensions. Let's say I am interested in using the IPCC2013 100 years method:

In [31]:
[m for m in bw.methods if "IPCC" in str(m) and ("2013") in str(m) and "100" in str(m)]

[('IPCC 2013 no LT', 'climate change', 'GTP 100a'),
 ('IPCC 2013 no LT', 'climate change', 'GWP 100a'),
 ('IPCC 2013', 'climate change', 'GTP 100a'),
 ('IPCC 2013', 'climate change', 'GWP 100a')]

I am interested in the last of these, and will assign it to a variable

In [32]:
ipcc2013_name = [m for m in bw.methods if "IPCC" in str(m) and ("2013") in str(m) and "100" in str(m)][2]
ipcc2013_name

('IPCC 2013', 'climate change', 'GTP 100a')

In [33]:
type(ipcc2013_name)

tuple

In [34]:
ipcc_2013_method = bw.Method(ipcc2013_name)

In [35]:
type(ipcc_2013_method)

bw2data.method.Method

Again, there are a bunch of methods associated with a method object. You can access these by typing ipcc_2013_method. and clicking tab.  
For example, metadata:

In [36]:
ipcc_2013_method.name

('IPCC 2013', 'climate change', 'GTP 100a')

In [37]:
ipcc_2013_method.metadata

{'abbreviation': 'ipcc-2013cg.855daa796b755d97aa0c7a7c5444642e',
 'description': '',
 'filename': 'LCIA_implementation_3.3.xlsx',
 'num_cfs': 206,
 'unit': 'kg CO2-Eq'}

In [38]:
ipcc_2013_method.metadata['unit']

'kg CO2-Eq'

Let's use the `load` method to see what is in the object:

In [39]:
ipcc_2013_method.load()

[(('biosphere3', 'e259263c-d1f1-449f-bb9b-73c6d0a32a00'), 1.0),
 (('biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd'), 1.0),
 (('biosphere3', 'aa7cac3a-3625-41d4-bc54-33e2cf11ec46'), 1.0),
 (('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), 1.0),
 (('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'), 1.0),
 (('biosphere3', 'e1c597cc-14cb-4ebb-af07-7a93a5b77d34'), 1.0),
 (('biosphere3', '6d89125e-e9b7-4d7e-a1fc-ada45dbd8815'), 1.0),
 (('biosphere3', '78eb1859-abd9-44c6-9ce3-f3b5b33d619c'), 1.0),
 (('biosphere3', 'e4e9febc-07c1-403d-8d3a-6707bb4d96e6'), 1.0),
 (('biosphere3', 'e8787b5e-d927-446d-81a9-f56977bbfeb4'), 1.0),
 (('biosphere3', '259cf8d6-6ea8-4ccf-84b7-23c930a5b2b3'), -1.0),
 (('biosphere3', '8ae4d8bb-3e4b-4825-8325-94696d7a64fd'), -1.0),
 (('biosphere3', '60d424f7-d5a9-4549-9540-da06684bc3bb'), -1.0),
 (('biosphere3', '375bc95e-6596-4aa1-9716-80ff51b9da77'), -1.0),
 (('biosphere3', '49c594c6-b6a8-42a3-95c5-cca812fda80b'), 1.9578),
 (('biosphere3', 'a3beb5ac-5149-4

This contains tuples with (elementary flow, characterization factors). This can be made more human readable by using the `bw.get_activity`, the 'name' `key` and a list comprehension. Try it:

In [40]:
# [(Some code to generate a valid activity key for each elementary flow) for ef in ipcc_2013_method.load()]

In [ ]:
# Solution:
[(get_activity(ef[0])['name'], ef[1]) for ef in ipcc_2013_method.load()]

Enough said for now about methods.

### LCI datases

There is much information on the structure of LCI databases in Brightway2 [here](https://docs.brightwaylca.org/intro.html#inventory-databases), [here](http://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/Databases.ipynb) and [here](https://docs.brightwaylca.org/technical/bw2data.html#databases).  Probably the easiest way to learn about them, however, is to import one and have a look.  

Here is the code to import the ecoinvent v3.3 database. Don't do it though, not now: it takes too long, we're better off working with a smaller database and getting more done.

In [ ]:
fpei33 = "enter your path to the datasets e.g. r'C:\FolderWith\LotsOf\ecoSpold2\Files'"

Next, execute the following code. It will import the ecoinvent database. We will look in detail in another Notebook at what this code actually does.

In [44]:
if 'ecoinvent 3.3 cutoff' in bw.databases:
    print("Database has already been imported")
else:
    ei33 = bw.SingleOutputEcospold2Importer(fpei33, 'ecoinvent 3.3 cutoff')
    ei33.apply_strategies()
    ei33.statistics()

Database has already been imported


The database needs to be written to disk:

In [ ]:
ei33.write_database()

Let's instead import **ecoinvent v2.2**:

In [ ]:
fpei22 =  "enter your path to the datasets e.g. r'C:\FolderWith\LotsOf\ecoSpold2\Files'"

In [ ]:
if 'ecoinvent 2.2 cutoff' in bw.databases:
    print("Database has already been imported")
else:
    ei22 = SingleOutputEcospold1Importer(fpei22, 'ecoinvent 2.2')
    ei22.apply_strategies()
    ei22.statistics()

In [ ]:
ei22.write_database()

Other code to import LCI databases in other formats are found [here](https://bitbucket.org/cmutel/brightway2-io/src/211f748e7b9987aef452a1ead1f483cc0b4bc25c/bw2io/importers/?at=default). We'll explore this later.

If you check that the database has actually been added to your project: 

In [45]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 2.2
	ecoinvent 3.3 cutoff

Note: the ei22 (or ei33) object created above is not the actual database, but actually an object used strictly for importing. 

In [ ]:
type(ei22)

To access the actual database, you need to use the Database method: 

In [46]:
#Uncomment the one you actually imported. 

bw.Database('ecoinvent 2.2')
# bw.Database('ecoinvent 3.3 cutoff')

Brightway2 SQLiteBackend: ecoinvent 2.2

This is a more advanced topic, but note that there are alternative backends. See [here](https://docs.brightwaylca.org/technical/bw2data.html#inventory-data-backends).

Let's assign the database to a variable and see what we can do:

In [48]:
#Uncomment the one you actually imported. 

eidb = bw.Database('ecoinvent 2.2')
# eidb = bw.Database('ecoinvent 3.3 cutoff')

In [49]:
# Check the length of the database:
len(eidb)

4087

Again, we can get an idea of useful methods and attributes by typing eidb. and Tab. Do this now.

In [ ]:
eidb. #Press tab!

One can again load the entire database to iterate over activities or exchanges within the database. This is quite a big object, but your computers can take it.

In [50]:
eidb_loaded = eidb.load()

However, you often will not need to do that at all. The most common interaction with the database object is to access activities, add activities, save, etc. although this really depends on what you are doing with Brightway2...

#### Activities and exchanges

In the context of LCI databases, activities are the nodes "within the technosphere". They are therefore the columns in the technosphere matrix $A$.  
There are different ways to get access to an activity. Let's use the `random()` method for now to explore a rendom activity in the ecoinvent database.

In [51]:
random_act = eidb.random()

In [52]:
random_act

'drilling, CNC, brass' (kilogram, RER, ['metals', 'chipping'])

In [53]:
type(random_act)

bw2data.backends.peewee.proxies.Activity

To see what is stored in an activity object, let's convert our random act in a dictionary: 

In [54]:
random_act.as_dict()

{'authors': [{'address': 'Kanzleistrasse 4, 8610 Uster',
   'company': 'ESU',
   'country': 'CH',
   'email': 'esu-services@ecoinvent.org',
   'name': 'Roland Steiner'}],
 'categories': ['metals', 'chipping'],
 'code': '1d0e0446b60e0da5ee95be61a3c2fdbb',
 'comment': 'The reference for drilling is 1 kg of metal removed by drilling. As there is a large variation from factory to factory with regard to the LCI, it is advised that in case this dataset becomes important in the results, it has to be investigated further if the rough estimations made are applicable or not. \nThis dataset encompasses the direct electricity consumption of the machine. In the case of CNC drilling compressed air and lubricant oil (incl. disposal) are accounted for. Furthermore, the metal removed is already included. Machine as well as factory infrastructure and operation are considered as well. The metal removed is assumed to be recycled. Maintenance and tool (bit) replacements are not included. Degreasing is not 

Notice one important thing: no exchanges!  

**Exchanges are the edges between nodes.**  
These can be:  
  - an edge between two activities within the technosphere (an element $a_{ij}$ of matrix $A$)  
  - edges between an activity in the technosphere and an activity in the "biosphere" (an element $b_{kj}$ of the biosphere matrix $B$).

One can however iterate through the exchanges. At this point, it is actually the best way to get to an exchange:

In [55]:
# All exchanges:
[exc for exc in random_act.exchanges()]

[Exchange: 1.0 kilogram 'drilling, CNC, brass' (kilogram, RER, ['metals', 'chipping']) to 'drilling, CNC, brass' (kilogram, RER, ['metals', 'chipping'])>,
 Exchange: 0.0625 kilowatt hour 'electricity, low voltage, production UCTE, at grid' (kilowatt hour, UCTE, ['electricity', 'production mix']) to 'drilling, CNC, brass' (kilogram, RER, ['metals', 'chipping'])>,
 Exchange: 1.28 cubic meter 'compressed air, average installation, >30kW, 7 bar gauge, at supply network' (cubic meter, RER, ['mechanical engineering', 'compressed air supply']) to 'drilling, CNC, brass' (kilogram, RER, ['metals', 'chipping'])>,
 Exchange: 0.00382 kilogram 'lubricating oil, at plant' (kilogram, RER, ['chemicals', 'organics']) to 'drilling, CNC, brass' (kilogram, RER, ['metals', 'chipping'])>,
 Exchange: 0.00382 kilogram 'disposal, used mineral oil, 10% water, to hazardous waste incineration' (kilogram, CH, ['waste management', 'hazardous waste incineration']) to 'drilling, CNC, brass' (kilogram, RER, ['metals',

One could also decide to only iterate through the biosphere exchanges, biosphere exchanges or production exchanges using, respectively, `random_act.technosphere`, `random_act.biosphere` and `random_act.production`.  

In [ ]:
# Try this: 
# Biosphere exchanges (i.e. elementary flows)

# Production exchanges

# Technosphere exchanges

Let's look at one of these exchanges by assigning one to a variable and exploring it:

In [56]:
random_exchange = [exc for exc in random_act.exchanges()][2]

In [57]:
type(random_exchange)

bw2data.backends.peewee.proxies.Exchange

Again, you can have an idea of what is readily accessible in terms of methods and attributes by typing `random_exchange.` + Tab.

In [58]:
random_exchange. # Tab.

{'amount': 1.28,
 'categories': ('mechanical engineering', 'compressed air supply'),
 'comment': '(3,5,4,5,3,5); estimated',
 'input': ('ecoinvent 2.2', '4c133e9f80ad92007e3006fef6a0957b'),
 'loc': 0.24686007793152581,
 'location': 'RER',
 'name': 'compressed air, average installation, >30kW, 7 bar gauge, at supply network',
 'negative': False,
 'output': ('ecoinvent 2.2', '1d0e0446b60e0da5ee95be61a3c2fdbb'),
 'scale': 0.19602104388801175,
 'type': 'technosphere',
 'uncertainty type': 2,
 'unit': 'cubic meter'}

Let's see what makes up an exchange by converting our random exchange to a dictionary.

In [59]:
random_exchange.as_dict()

{'amount': 1.28,
 'categories': ('mechanical engineering', 'compressed air supply'),
 'comment': '(3,5,4,5,3,5); estimated',
 'input': ('ecoinvent 2.2', '4c133e9f80ad92007e3006fef6a0957b'),
 'loc': 0.24686007793152581,
 'location': 'RER',
 'name': 'compressed air, average installation, >30kW, 7 bar gauge, at supply network',
 'negative': False,
 'output': ('ecoinvent 2.2', '1d0e0446b60e0da5ee95be61a3c2fdbb'),
 'scale': 0.19602104388801175,
 'type': 'technosphere',
 'uncertainty type': 2,
 'unit': 'cubic meter'}

Of prime interest to identify the exchange:  
  - The `input` is the activity the exchange is originating from
  - The `output` is the activity the exchange is terminating in  
    - If `input` == `output`, the exchange is actually the reference flow (production exchange)  
    - If the exchange is a "biosphere exchange" (i.e. an elementary flow), then its `output` will be in the biosphere database.

#### Our first LCA!

Brightway has a so-called LCA object. It is instantiated using `LCA(args)`. The only required argument is a functional unit, described by a dictionary with keys = activities and values = amounts (more [here](https://docs.brightwaylca.org/lca.html#specifying-a-functional-unit)). A second argument that is often passed is an LCIA method, passed using the method tuple.  
Let's create our first LCA object using our random activity and our IPCC method.  

In [148]:
myFirstLCA_quick = bw.LCA({random_act:1}, ('IPCC 2013', 'climate change', 'GWP 100a'))

The steps to get to the impact score are as follows:

In [149]:
myFirstLCA_quick.lci()    # Builds matrices, solves the system, generates an LCI matrix.
myFirstLCA_quick.lcia()   # Characterization, i.e. the multiplication of the elements 
                          # of the LCI matrix with characterization factors from the chosen method
myFirstLCA_quick.score    # Returns the score, i.e. the sum of the characterized inventory

4.090895588283267

Let's do this again, but not so quickly now, so we can see what is going on:

In [61]:
myFirstLCA = bw.LCA({random_act:1}, ('IPCC 2013', 'climate change', 'GWP 100a'))

We can now explore the methods and properties of the LCA object:

In [ ]:
myFirstLCA. #Press Tab

Let's explore a few:

#### Demand

In [62]:
myFirstLCA.demand

{'drilling, CNC, brass' (kilogram, RER, ['metals', 'chipping']): 1}

To access the actual activity from the demand, you would do this:

In [63]:
list(myFirstLCA.demand.keys())[0]

'drilling, CNC, brass' (kilogram, RER, ['metals', 'chipping'])

In [64]:
demanded_act = list(myFirstLCA.demand.keys())[0]

In [65]:
demanded_act == random_act

True

There are also other attributes that have simply not been built yet: 

In [66]:
myFirstLCA.demand_array

AttributeError: 'LCA' object has no attribute 'demand_array'

In [67]:
myFirstLCA.score

AssertionError: Must do LCIA first

This is because the actual matrices have not yet been built. Running myFirstLCA.lci() will:
 - attribute row and column numbers to all elements in our $A$ and $B$ matrices and store these in a paramerer array (NumPy structured array) - processed data.
 - Build coordinate (coo) matrices based on this information - actual matrices.  
 
The turning of the processed data (structured arrays) into matrices is described [here](https://docs.brightwaylca.org/lca.html#building-matrices).  

Let's run the lci() method now:

In [68]:
myFirstLCA.lci()

Now we have access to many other attributes and methods.

**Demand array**, the $f$ in $As=f$

The demand array is a numpy array, where all elements are = 0 except for the ones specified in the functional unit.

In [69]:
myFirstLCA.demand_array

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [70]:
type(myFirstLCA.demand_array)

numpy.ndarray

In [71]:
myFirstLCA.demand_array.shape

(4087,)

In [72]:
myFirstLCA.demand_array.sum()

1.0

So where is this "1"? This is where we need to start talking about **indices**. The row and column indices are stored in LCA-specific dictionaries. For example, we have product_dict that links the types of products used in our LCA (rows in the $A$ matrix) with the row numbers in the actual $A$ matrix that Brightway built.

In [73]:
myFirstLCA.product_dict

{('ecoinvent 2.2', '347c428c1e4df89f46e66ebde736ea32'): 4016,
 ('ecoinvent 2.2', '6b35b48a0e398444d8b8c18fe77d62b0'): 3021,
 ('ecoinvent 2.2', 'f5ce334466231adc4167dd2d025f065e'): 1662,
 ('ecoinvent 2.2', '99d2906d9533b8a191e280f0aa98a107'): 592,
 ('ecoinvent 2.2', '54520e53a4c15660f19ad6a50890bf2d'): 2981,
 ('ecoinvent 2.2', '0d79f0364cb44e94ff00c5cd4e057727'): 52,
 ('ecoinvent 2.2', '6068694945876a175a3c75c094e5197a'): 538,
 ('ecoinvent 2.2', '98eb4b1cfee2d1715f3157f409e26d5e'): 1951,
 ('ecoinvent 2.2', '24b0d02a6551fee1b9120a3f0eeb81b5'): 1228,
 ('ecoinvent 2.2', '926eb37d323d9c0577f080d547e2eac9'): 1830,
 ('ecoinvent 2.2', 'cc2741cc570cc269d1da89d3eae71427'): 871,
 ('ecoinvent 2.2', '9c407322be79bebf989374a3bf6f10f3'): 1114,
 ('ecoinvent 2.2', '34152e268b72e615bbff6405f4606656'): 2486,
 ('ecoinvent 2.2', '4fda80619cf844945e1ed4384a66f87c'): 1526,
 ('ecoinvent 2.2', '0839fd914788aed5aaac9e31a50dfca7'): 2768,
 ('ecoinvent 2.2', '36686269d40955216a062f1db1223655'): 2568,
 ('ecoinvent 

There are three such dictionaries: 
 - `activity_dict`: Columns in the technosphere matrix $A$ or biosphere matrix $B$
 - `product_dict` : Rows in the technosphere matrix $A$  
 - `biosphere_dict`: Rows in the biosphere matrix $B$

In passing, note that our (square) $A$ matrix has the same row and column dimensions, in other words:

In [74]:
myFirstLCA.activity_dict == myFirstLCA.product_dict

True

So, to our question (where is this "1"), we need to use the `activity_dict` to find out. The key we need is the (database, code) tuple of our demand:

In [75]:
demand_database = list(myFirstLCA.demand.keys())[0]['database']
demand_code = list(myFirstLCA.demand.keys())[0]['code']
(demand_database, demand_code)

('ecoinvent 2.2', '1d0e0446b60e0da5ee95be61a3c2fdbb')

In [76]:
row_of_demand = myFirstLCA.activity_dict[(demand_database, demand_code)]
row_of_demand # Row number of our demand vector containing the functional unit.

4042

In [77]:
myFirstLCA.demand_array[row_of_demand]

1.0

The .lci() also created other very important arrays:

** $A$ matrix** (technology matrix), with each element $a_{ij}$ provides information on the amount of input or output of product $i$ from activity $j$. When $i=j$, the element $a_{ij}$ is the *reference flow* for the activity described in the column.

In [78]:
myFirstLCA.technosphere_matrix

<4087x4087 sparse matrix of type '<class 'numpy.float64'>'
	with 43045 stored elements in Compressed Sparse Row format>

It is a `sparse matrix` in Compressed Sparse Row format.  
It has three "parts":  
  - an array of column indices, `.indices`  
  - an array of points to row starts in indices and data, `.indptr`  
  - an array with the actual value, `.data`  
Printing it gives an overview of what that looks like (although in the print-out the rows are no longer *compressed* per se):

In [79]:
print(myFirstLCA.technosphere_matrix)

  (0, 0)	1.0
  (0, 2288)	-0.629999995232
  (0, 2322)	-1300.0
  (0, 2480)	-6.79479980469
  (0, 2883)	-281.359985352
  (1, 1)	1.0
  (2, 2)	1.0
  (2, 1038)	-3.78999999787e-12
  (3, 3)	1.0
  (3, 140)	-0.00435800012201
  (3, 158)	-0.00435800012201
  (3, 190)	-0.00258040009066
  (3, 197)	-0.0245140008628
  (3, 555)	-0.00435800012201
  (3, 639)	-0.0245140008628
  (3, 968)	-0.00435800012201
  (3, 1704)	-0.00258040009066
  (3, 1820)	-0.00435800012201
  (3, 2034)	-0.00435800012201
  (3, 2148)	-0.00435800012201
  (3, 2241)	-0.0245140008628
  (3, 2696)	-0.0245140008628
  (3, 3116)	-0.0245140008628
  (3, 3181)	-0.0245140008628
  (3, 3260)	-0.00435800012201
  :	:
  (4080, 899)	-4.26569997103e-08
  (4080, 1342)	-1.26049997107e-08
  (4080, 3080)	-1.89070004097e-09
  (4080, 3283)	-2.40889992398e-08
  (4080, 4080)	1.0
  (4081, 2488)	-1.13639998436
  (4081, 4081)	1.0
  (4082, 4082)	1.0
  (4083, 9)	-0.00447000004351
  (4083, 902)	-0.00499999988824
  (4083, 1078)	-0.00499999988824
  (4083, 2305)	-0.0219999

We can slice this matrix using coordinates. For example, let's say we wanted a view of the exchanges associated with the unit process providing our functional unit.  
We already know that the `key` for this activity is `(demand_database, demand_code)`, constructed above:

In [159]:
(demand_database, demand_code)

('ecoinvent 2.2', '1d0e0446b60e0da5ee95be61a3c2fdbb')

We also know that the column number for this activity in the $A$ matrix is stored in the `activity_dict`

In [160]:
myFirstLCA.activity_dict[(demand_database, demand_code)]

4042

So we can return a one-dimensional matrix representing the column in the $A$ matrix that supplies our functional unit: 

In [162]:
myColumn = myFirstLCA.technosphere_matrix[:, myFirstLCA.activity_dict[(demand_database, demand_code)]]
myColumn

<4087x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

Printing this out gives:

In [164]:
print(myColumn)

  (26, 0)	-0.00381999998353
  (1188, 0)	-0.00381999998353
  (1306, 0)	-1.0
  (1787, 0)	-3.94999988202e-05
  (2062, 0)	-2.01999994509e-09
  (2398, 0)	-1.27999997139
  (2566, 0)	-4.40999984741
  (3376, 0)	-0.0625
  (4042, 0)	1.0


Not too useful: it would be better to get the *names* to these exchanges.  
We need to do two things:  
  - Get the indices from the CSR matrix (we can do this by converting it to a sparse matrix in `COOrdinate` format first)  
  - Get the activity code for the each index (we can do this using the reverse of the `activity_dict`)  
  - Use `get_activity` to access the actual names of the activities.  

1) Converting the CSR matrix to a COO matrix:  

In [179]:
myColumnCOO = myColumn.tocoo()
myColumnCOO

<4087x1 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in COOrdinate format>

It is still a sparse matrix with the same number of elements, and it looks quite like the CSR version when we print it out:

In [180]:
print(myColumnCOO)

  (26, 0)	-0.00381999998353
  (1188, 0)	-0.00381999998353
  (1306, 0)	-1.0
  (1787, 0)	-3.94999988202e-05
  (2062, 0)	-2.01999994509e-09
  (2398, 0)	-1.27999997139
  (2566, 0)	-4.40999984741
  (3376, 0)	-0.0625
  (4042, 0)	1.0


However, we can directly access the rows and column indices using `row` and `col`:  

In [182]:
myColumnCOO.row

array([  26, 1188, 1306, 1787, 2062, 2398, 2566, 3376, 4042])

2) Get the activity code for each element  
We have a dictionary that gives us the indices of activities/products, but now we need the opposite: we need a dictionary that will give us the activity code for a given index. We therefore need to create a *reverse* dictionary, which we can do as follows:

In [183]:
myFirstLCA_reverse_activity_dict = {v:k for k, v in myFirstLCA.activity_dict.items()}
myFirstLCA_reverse_activity_dict

{0: ('ecoinvent 2.2', '8609adf0a083959f2524cd2aad891f51'),
 1: ('ecoinvent 2.2', '31894eb0fc23d68e1d65e66771c7f014'),
 2: ('ecoinvent 2.2', '09d2c123a37ca00b3e1582d4dc35994f'),
 3: ('ecoinvent 2.2', '4bc630232ab2ad4f025dc2d1d1ce735c'),
 4: ('ecoinvent 2.2', 'd705bd6f5250730edd0a4bab1a04c7de'),
 5: ('ecoinvent 2.2', '98150d2c16832da415d7819749d03fbc'),
 6: ('ecoinvent 2.2', '18212e116c3223b74f1e6e575719d24d'),
 7: ('ecoinvent 2.2', '66d83b77dff5d93199c5a7092c0d5a34'),
 8: ('ecoinvent 2.2', '7af9f6c24f783bcc241acbe4bdf9bd98'),
 9: ('ecoinvent 2.2', 'bd7b94cec97b65332b71c00c3d9af123'),
 10: ('ecoinvent 2.2', '119ca61a751dd1d929ac1bf5231e7958'),
 11: ('ecoinvent 2.2', '2a5a71c124b6b49ac3eafa584574a096'),
 12: ('ecoinvent 2.2', 'ecbe970f2a688469972e2133197af640'),
 13: ('ecoinvent 2.2', '0642b64c2ff903f46436eda35a62f508'),
 14: ('ecoinvent 2.2', '973d538466133bbff4eff0dc9b3d863c'),
 15: ('ecoinvent 2.2', 'cb15be9b4cb31e5b4e024d3df5d629e5'),
 16: ('ecoinvent 2.2', '60a90384ea4a6b5ca542bed757

We can therefore now create a list of the different activities with which our column exchanges:  

In [190]:
[myFirstLCA_reverse_activity_dict[i] for i in myColumnCOO.row]

[('ecoinvent 2.2', '64f6d365488c965e1293b3d612fd5146'),
 ('ecoinvent 2.2', '00623899611a18f8616b6d2d9de7adfb'),
 ('ecoinvent 2.2', '73629957c7ab55f6aa2adeded1c64c1c'),
 ('ecoinvent 2.2', 'd99b7a72a9ec9559d6008bd1ebd87030'),
 ('ecoinvent 2.2', '93e78a2693829d6981838c4a4c367ec6'),
 ('ecoinvent 2.2', '4c133e9f80ad92007e3006fef6a0957b'),
 ('ecoinvent 2.2', 'f43ec77add94eae309c8f742b9da877d'),
 ('ecoinvent 2.2', 'ec263f3ac8e4fd29f369b4b0d50be575'),
 ('ecoinvent 2.2', '1d0e0446b60e0da5ee95be61a3c2fdbb')]

It would be even nicer to get the names for these:

In [193]:
[bw.get_activity(myFirstLCA_reverse_activity_dict[i])['name'] for i in myColumnCOO.row]

['disposal, used mineral oil, 10% water, to hazardous waste incineration',
 'lubricating oil, at plant',
 'brass, at plant',
 'metal working machine, unspecified, at plant',
 'metal working factory',
 'compressed air, average installation, >30kW, 7 bar gauge, at supply network',
 'metal working factory operation, average heat energy',
 'electricity, low voltage, production UCTE, at grid',
 'drilling, CNC, brass']

We can put these in a neat Pandas Series, with actual names and amounts:

In [211]:
# First create a dict with the information I want:
myColumnAsDict = dict(zip(
        [bw.get_activity(myFirstLCA_reverse_activity_dict[i])['name'] for i in myColumnCOO.row],
        myColumnCOO.data)
                      )
# Create Pandas Series from dict
pd.Series(myColumnAsDict, name="Nice series with information on exchanges in my foreground process")

brass, at plant                                                               -1.000000e+00
compressed air, average installation, >30kW, 7 bar gauge, at supply network   -1.280000e+00
disposal, used mineral oil, 10% water, to hazardous waste incineration        -3.820000e-03
drilling, CNC, brass                                                           1.000000e+00
electricity, low voltage, production UCTE, at grid                            -6.250000e-02
lubricating oil, at plant                                                     -3.820000e-03
metal working factory                                                         -2.020000e-09
metal working factory operation, average heat energy                          -4.410000e+00
metal working machine, unspecified, at plant                                  -3.950000e-05
Name: Nice series with information on exchanges in my foreground process, dtype: float64

**$B$ matrix**, with each element $b_{kj}$ provides information on the amount of input or output of elementary flow $b_k$  from activity $j$.

In [80]:
myFirstLCA.biosphere_matrix

<1584x4087 sparse matrix of type '<class 'numpy.float64'>'
	with 92000 stored elements in Compressed Sparse Row format>

It is again a CSR sparse matrix.

Exercise: Create a Pandas series with the emissions of elementary flow 'Carbon dioxide, fossil' (kilogram, None, ('air',), per unit process (before solving the system).

Solution:

In [213]:
# Find the key to the elementary flow:
elementary_flow = [ef for ef in bw.Database('biosphere3') if "'Carbon dioxide, fossil' (kilogram, None, ('air',)" in str(ef)][0]
elementary_flow

'Carbon dioxide, fossil' (kilogram, None, ('air',))

In [217]:
index = myFirstLCA.biosphere_dict[('biosphere3', elementary_flow['code'])]
index

559

In [225]:
myFirstLCA.biosphere_matrix.tocoo().col

array([  51,   71,   74, ..., 3519, 3864, 4004], dtype=int32)

In [228]:
# Get the row from the matrix
CO2_emissions = myFirstLCA.biosphere_matrix[index, :]
# Convert it to an array
CO2_emissions = CO2_emissions.toarray().flatten()
# Generate a list of activity names for each column
activity_names = [bw.get_activity(myFirstLCA_reverse_activity_dict[j])['name'] for j in myFirstLCA.biosphere_matrix.tocoo().col]
# Generate a Series:
pd.Series(dict(zip(activity_names, CO2_emissions)), name="CO2 emissions, per unit process, in B matrix").sort_values(ascending=False)

transport, natural gas, pipeline, long distance                                     928.669983
disposal, LCD module,  to municipal waste incineration                              633.000000
wood chips, from industry, hardwood, burned in furnace 300kW                        580.630005
silver, from combined gold-silver production, at refinery                           284.869995
operation, lorry 7.5-16t, EURO5                                                     157.960007
polystyrene, general purpose, GPPS, at plant                                         35.419998
fungicides, at regional storehouse                                                    1.104500
disposal, anion exchange resin f. water, 50% water, to municipal incineration         1.103400
drying, natural gas                                                                   0.848220
jute stalks, from fibre production, irrigated system, at farm                         0.692330
monoammonium phosphate, as N, at regional storehou

This is actually a quite slow way to do it: better to use db_loaded...

In [229]:
# TO DO...

**Supply matrix**: Vector containing the amount each activity will need to provide to meet the functional demand, i.e. $s=A^{-1}f$.

In [82]:
myFirstLCA.supply_array

array([  1.77019883e-02,   0.00000000e+00,   1.34047711e-12, ...,
         0.00000000e+00,   3.01343875e-10,   0.00000000e+00])

**Inventory matrix**: Contains the inventory *by activity* (i.e. not summed). In other words, we do not have $g=BA^{-1}f$, but rather $G=B diag(A^{-1}f)$

In [150]:
myFirstLCA.inventory

<1584x4087 sparse matrix of type '<class 'numpy.float64'>'
	with 50231 stored elements in Compressed Sparse Row format>

In [151]:
myFirstLCA.inventory.indptr

array([    0,    45,    58, ..., 50194, 50219, 50231], dtype=int32)

In [152]:
myFirstLCA.inventory.indices

array([4004, 3864, 3848, ..., 1301,  957,  710], dtype=int32)

In [153]:
myFirstLCA.inventory.data

array([  1.12567342e-15,   1.70728727e-13,   9.60963721e-16, ...,
         1.35677042e-14,   3.33407514e-11,   1.67356817e-14])

We can look at this matrix by storing it in a 

In [131]:
myFirstLCA.inventory.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [130]:
pd.DataFrame(myFirstLCA.inventory.toarray()).head(3)

,0,1,2,3,4,5,6,7,8,9,...,4077,4078,4079,4080,4081,4082,4083,4084,4085,4086
0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,3.666096e-15,0.0,0.0,3.249256e-11,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
print(myFirstLCA.inventory)

  (0, 4004)	1.12567341965e-15
  (0, 3864)	1.70728727437e-13
  (0, 3848)	9.60963720997e-16
  (0, 3841)	8.31696700832e-16
  (0, 3817)	5.39798080518e-22
  (0, 3789)	1.62826612912e-11
  (0, 3737)	1.06304300772e-15
  (0, 3666)	4.16149201675e-13
  (0, 3630)	9.95214594355e-12
  (0, 3604)	7.66906144939e-18
  (0, 3519)	1.69276837471e-15
  (0, 3414)	8.64446165297e-19
  (0, 3129)	5.09974491734e-13
  (0, 3049)	1.19705555293e-12
  (0, 3014)	1.11424676673e-11
  (0, 2848)	1.22497780619e-17
  (0, 2772)	1.66627887584e-14
  (0, 2730)	1.01223668536e-11
  (0, 2649)	2.21095561615e-14
  (0, 2639)	9.49222033136e-17
  (0, 2562)	3.17702639645e-14
  (0, 2540)	1.24852484522e-08
  (0, 2513)	1.02294564513e-11
  (0, 2454)	3.89930739511e-09
  (0, 2429)	2.59480956525e-14
  :	:
  (1582, 1905)	9.42754501698e-14
  (1582, 1834)	1.49145983641e-10
  (1582, 1301)	1.22589526578e-16
  (1582, 1288)	8.19207779718e-10
  (1582, 1105)	2.37527790991e-10
  (1582, 1021)	3.48673604883e-12
  (1582, 957)	7.41893923473e-13
  (1582, 770)	

tech_params, bioparams
The matrices above only get populated with numbers. However, behind these are the structured arrays mentioned above. These are also accessible:

In [85]:
import pandas as pd

In [86]:
pd.DataFrame(myFirstLCA.tech_params).head(2)

,input,output,row,col,type,uncertainty_type,amount,loc,scale,shape,minimum,maximum,negative
0,17861,17861,0,0,0,0,1.0,1.0,NaN,NaN,NaN,NaN,False
1,17862,17862,1,1,0,0,1.0,1.0,NaN,NaN,NaN,NaN,False


In [125]:
pd.DataFrame(myFirstLCA.bio_params).head(2)

,input,output,row,col,type,uncertainty_type,amount,loc,scale,shape,minimum,maximum,negative
0,3,17912,0,51,2,2,6.005100e-03,-5.115146,0.808276,NaN,NaN,NaN,False
1,3,17932,0,71,2,2,4.151800e-07,-14.694553,0.231867,NaN,NaN,NaN,False


We can manually aggregate the LCI if we want:

In [89]:
myFirstLCA.inventory.shape

(1584, 4087)

In [90]:
LCI_summed = myFirstLCA.inventory.sum(axis=1)

Again, to identify what number corresponds to what, you need to use the biosphere_dict or its reverse (see [here](http://stackoverflow.com/questions/39494583/connecting-exchange-names-and-codes-to-lca-inventory-results/39518156#39518156))

Next step: **LCIA**

In [91]:
myFirstLCA.lcia()

A number of other matrices are now available:

In [92]:
myFirstLCA.characterization_matrix

<1584x1584 sparse matrix of type '<class 'numpy.float64'>'
	with 67 stored elements in Compressed Sparse Row format>

In [93]:
myFirstLCA.characterization_matrix.shape

(1584, 1584)

In [94]:
myFirstLCA.characterized_inventory

<1584x4087 sparse matrix of type '<class 'numpy.float64'>'
	with 2857 stored elements in Compressed Sparse Row format>

The overall score is now an attribute of the LCA object: 

In [95]:
myFirstLCA.score

4.090895588283267

We also could have determined what this score was by summing the elements of our `characterized_inventory` matrix:

In [97]:
myFirstLCA.characterized_inventory.sum()

4.0908955882832672

We could also calculate the score by elementary flow (summing columns for each rows), irrespective of the unit process that produced it:

In [109]:
elementary_flow_contribution = myFirstLCA.characterized_inventory.sum(axis=1) #Axis is the dimension I want to sum over:

In [112]:
elementary_flow_contribution.shape

(1584, 1)

Notice that is has **two** dimensions. The result is in fact a one-dimensional matrix:

In [111]:
type(elementary_flow_contribution)

numpy.matrixlib.defmatrix.matrix

To convert it to an array (probably more useful for many purposes), you can use any of the following approaches:

In [115]:
import numpy as np

In [124]:
elementary_flow_contribution.A1 
#np.squeeze(np.asarray(elementary_flow_contribution))
#np.asarray(elementary_flow_contribution).reshape(-1)
#np.array(elementary_flow_contribution).flatten()
#np.array(elementary_flow_contribution).ravel()

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

Excercise: Create a Pandas series that has the scores per unit process, sorted by value (contribution analysis)